# Этап 3: Построение признаков для классификации доступности

Этот ноутбук выполняет построение признаков для задачи классификации: доступен ли медицинский объект в пределах заданного расстояния от спортивного объекта.

In [5]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point
from shapely.ops import nearest_points
from tqdm import tqdm
tqdm.pandas()

In [6]:
# Загрузка спортивных объектов с маршрутами
sport_with_routes = gpd.read_file("../data/processed/sport_with_routes.geojson")

# Объединение всех мед. учреждений
med_files = [
    "med_1258", "med_517", "med_502", "med_516", "med_1260"
]
med_gdfs = [gpd.read_file(f"../data/processed/{name}.geojson") for name in med_files]
med = pd.concat(med_gdfs, ignore_index=True).pipe(gpd.GeoDataFrame, geometry="geometry", crs="EPSG:4326")

print(f"[INFO] Всего мед. объектов: {len(med)}")

[INFO] Всего мед. объектов: 167


In [7]:
# Перевод в метры (EPSG:3857)
sport_proj = sport_with_routes.to_crs(epsg=3857)
med_proj = med.to_crs(epsg=3857)

# Функция поиска ближайшего расстояния
def find_nearest_distance(point, candidates):
    nearest = candidates.geometry.distance(point).min()
    return nearest

# Считаем расстояние до ближайшего мед. объекта
sport_proj["distance_to_med"] = sport_proj.geometry.progress_apply(lambda x: find_nearest_distance(x, med_proj))

# Бинарная целевая переменная
threshold_meters = 1500
sport_proj["accessible"] = (sport_proj["distance_to_med"] <= threshold_meters).astype(int)


100%|██████████| 13181/13181 [00:00<00:00, 17084.97it/s]


In [8]:
# Количество маршрутов рядом
features = sport_proj.groupby("stop_global_id").agg({
    "route_number": "nunique",
    "transport_type": lambda x: x.value_counts().to_dict()
}).rename(columns={"route_number": "num_routes"})

# Распаковка словаря типов транспорта
features = features.join(
    features["transport_type"].apply(pd.Series).fillna(0).astype(int)
).drop(columns=["transport_type"])

# Объединяем обратно
sport_proj = sport_proj.join(features, on="stop_global_id", rsuffix="_agg")

# Удаляем дубликаты по спорту
sport_final = sport_proj.drop_duplicates(subset=["stop_global_id"]).copy()


In [10]:
# Финальная таблица
columns_to_keep = ["stop_global_id", "distance_to_med", "accessible", "num_routes"] +     [col for col in sport_final.columns if col not in ["geometry", "route_number", "transport_type"] and col not in ["stop_global_id", "distance_to_med", "accessible", "num_routes"]]

features_df = sport_final[["stop_global_id", "distance_to_med", "accessible", "num_routes"] + columns_to_keep]
features_df.to_csv("../data/processed/features.csv", index=False)

print(f"[INFO] Сохранено: data/processed/features.csv ({len(features_df)} объектов)")

[INFO] Сохранено: data/processed/features.csv (727 объектов)
